## Calculation of R parameter


default: 
- atmosphere = April
- nucleus = p
- hadronic interaction model = Sibyll2.3c


modification parameters:
- nucleus type
- atmosphere
- depth: 1.5 or 3.5
- pi-air cross section factor


In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm
import pickle
import numpy as np

import crflux.models as pm
import mceq_underground_helpers_cs as mh


In [2]:
plot_dir = '/hetghome/khymon/Plots/cs-analysis/'

In [3]:
d = 1.5 #1.5 or 3.5

In [4]:
s_d = mh.slant_depths
angles = mh.angles
x_mod = d/np.cos(np.deg2rad(angles)) # for specific depth

In [5]:
m = mh.n_mu_vec # muon multiplicity

In [6]:
def X(d):
    s_d = mh.slant_depths
    angles = mh.angles
 
    return d/np.cos(np.deg2rad(angles))
    

In [7]:
def dNmu_dmu(d,month, ptype, cs): # month = str
    angle = 0.0

    #get depth according to angle
    x_mod = X(d)

    #get number of muons per multiplicity
    dNmudmu = np.ndarray(shape=(len(x_mod),101),dtype='float')     
    for i in range(len(x_mod)):

        dNmudmu[i] = 1e4*mh.mult_dist(x_mod[i],
                                    0.,
                                    pm.GlobalSplineFitBeta(),"yields_" +month,
                                    ptype,
                                    cs,
                                    norm=False
                                ) / mh.rates(x_mod[i], angle, month, ptype, cs)
    return dNmudmu
    

In [8]:
# function to calculate R from dN_mu from unmodified and modofied cs 
def R(m,dN_dNmu):
    int_low = np.zeros(len(dN_dNmu[:,0]))
    int_high = np.zeros(len(dN_dNmu[:,0]))

    for i in range(len(dN_dNmu[:,0])):
        int_low[i] = np.trapezoid(dN_dNmu[i,1:19],m[1:19])
        int_high[i] = np.trapezoid(dN_dNmu[i,59:],m[59:])    

    return int_high / int_low # return as array for x or angle



In [23]:
def R_normalized(R_mod,d,ptype):
    # default parameters
    dNu_dmu_apr = dNmu_dmu(d,month="apr", ptype=ptype, cs=1.0) #default cs
    print(dNu_dmu_apr)
    Rdef_apr = R(m,dNu_dmu_apr)
    print(Rdef_apr)
    return R_mod/Rdef_apr

In [24]:
dNmu_dmu(d,month="apr", ptype=2212,cs=1.0)

array([[2.19980476e-02, 2.17814397e-03, 5.40513771e-04, 2.00739634e-04,
        9.16365494e-05, 4.85247674e-05, 2.82739988e-05, 1.77356945e-05,
        1.16347440e-05, 7.99621418e-06, 5.64765843e-06, 4.11691006e-06,
        3.04996939e-06, 2.31860487e-06, 1.78271808e-06, 1.39766393e-06,
        1.10927498e-06, 8.85491857e-07, 7.18615491e-07, 5.88618073e-07,
        4.86263061e-07, 4.02640943e-07, 3.38012132e-07, 2.85672126e-07,
        2.42955946e-07, 2.07087220e-07, 1.78221417e-07, 1.54171294e-07,
        1.34015052e-07, 1.17028383e-07, 1.02233400e-07, 9.00688803e-08,
        7.96598653e-08, 7.07136873e-08, 6.29925590e-08, 5.63022059e-08,
        5.04830639e-08, 4.51123464e-08, 4.07156676e-08, 3.68495155e-08,
        3.34392030e-08, 3.04216369e-08, 2.77434305e-08, 2.53594896e-08,
        2.31519391e-08, 2.12592598e-08, 1.95606091e-08, 1.80323859e-08,
        1.66542831e-08, 1.54087721e-08, 1.42806761e-08, 1.32568154e-08,
        1.23257125e-08, 1.14773447e-08, 1.06674657e-08, 9.963626

In [25]:
R(m,dNmu_dmu(d,month="apr", ptype=2212,cs=1.0))

array([6.25745447e-05, 6.96276325e-05, 7.79682006e-05])

In [26]:
dNu_dmu_apr=dNmu_dmu(d,month="apr",ptype=2212,cs=1.0)
dNu_dmu_jan = dNmu_dmu(d,month="jan", ptype=2212, cs=1.0)

In [13]:

dNu_dmu_jan_mod = dNmu_dmu(d,month="jan", ptype=2212, cs=1.5)

In [14]:
np.shape(dNu_dmu_jan)

(3, 101)

In [15]:
x_mod

array([1.875     , 1.66666667, 1.5       ])

In [16]:
dNu_dmu_jan[:,1:19]

array([[2.40927330e-03, 5.93073585e-04, 2.18638573e-04, 9.99705295e-05,
        5.24652171e-05, 3.04549746e-05, 1.89546725e-05, 1.24004702e-05,
        8.51263826e-06, 6.00137063e-06, 4.37151782e-06, 3.23304549e-06,
        2.45670141e-06, 1.88655848e-06, 1.47876082e-06, 1.17337039e-06,
        9.35515688e-07, 7.58962741e-07],
       [3.98636803e-03, 9.89519583e-04, 3.62975099e-04, 1.67419127e-04,
        8.83965591e-05, 5.12219650e-05, 3.20136408e-05, 2.12367113e-05,
        1.46123956e-05, 1.04483291e-05, 7.62613802e-06, 5.73326897e-06,
        4.36786548e-06, 3.40374544e-06, 2.68968240e-06, 2.13784721e-06,
        1.73115947e-06, 1.41543355e-06],
       [5.99485012e-03, 1.49419513e-03, 5.55338299e-04, 2.53835875e-04,
        1.34896294e-04, 7.86191093e-05, 4.94413632e-05, 3.25558509e-05,
        2.24585032e-05, 1.61271331e-05, 1.18421485e-05, 8.95093866e-06,
        6.84657071e-06, 5.36783496e-06, 4.26762033e-06, 3.41529309e-06,
        2.78151623e-06, 2.28731360e-06]])

In [27]:

R_def_apr = R(m,dNu_dmu_apr)
R_def_apr

array([6.25745447e-05, 6.96276325e-05, 7.79682006e-05])

In [28]:
R_def_apr/R_def_apr

array([1., 1., 1.])

In [30]:
dNmu_dmu(d,month="apr", ptype=2212,cs=1.0)

array([[2.19980476e-02, 2.17814397e-03, 5.40513771e-04, 2.00739634e-04,
        9.16365494e-05, 4.85247674e-05, 2.82739988e-05, 1.77356945e-05,
        1.16347440e-05, 7.99621418e-06, 5.64765843e-06, 4.11691006e-06,
        3.04996939e-06, 2.31860487e-06, 1.78271808e-06, 1.39766393e-06,
        1.10927498e-06, 8.85491857e-07, 7.18615491e-07, 5.88618073e-07,
        4.86263061e-07, 4.02640943e-07, 3.38012132e-07, 2.85672126e-07,
        2.42955946e-07, 2.07087220e-07, 1.78221417e-07, 1.54171294e-07,
        1.34015052e-07, 1.17028383e-07, 1.02233400e-07, 9.00688803e-08,
        7.96598653e-08, 7.07136873e-08, 6.29925590e-08, 5.63022059e-08,
        5.04830639e-08, 4.51123464e-08, 4.07156676e-08, 3.68495155e-08,
        3.34392030e-08, 3.04216369e-08, 2.77434305e-08, 2.53594896e-08,
        2.31519391e-08, 2.12592598e-08, 1.95606091e-08, 1.80323859e-08,
        1.66542831e-08, 1.54087721e-08, 1.42806761e-08, 1.32568154e-08,
        1.23257125e-08, 1.14773447e-08, 1.06674657e-08, 9.963626

In [31]:
R_normalized(R_def_apr,d,2212)

[[2.19980476e-02 2.17814397e-03 5.40513771e-04 2.00739634e-04
  9.16365494e-05 4.85247674e-05 2.82739988e-05 1.77356945e-05
  1.16347440e-05 7.99621418e-06 5.64765843e-06 4.11691006e-06
  3.04996939e-06 2.31860487e-06 1.78271808e-06 1.39766393e-06
  1.10927498e-06 8.85491857e-07 7.18615491e-07 5.88618073e-07
  4.86263061e-07 4.02640943e-07 3.38012132e-07 2.85672126e-07
  2.42955946e-07 2.07087220e-07 1.78221417e-07 1.54171294e-07
  1.34015052e-07 1.17028383e-07 1.02233400e-07 9.00688803e-08
  7.96598653e-08 7.07136873e-08 6.29925590e-08 5.63022059e-08
  5.04830639e-08 4.51123464e-08 4.07156676e-08 3.68495155e-08
  3.34392030e-08 3.04216369e-08 2.77434305e-08 2.53594896e-08
  2.31519391e-08 2.12592598e-08 1.95606091e-08 1.80323859e-08
  1.66542831e-08 1.54087721e-08 1.42806761e-08 1.32568154e-08
  1.23257125e-08 1.14773447e-08 1.06674657e-08 9.96362644e-09
  9.31877378e-09 8.72698876e-09 8.18303908e-09 7.68228683e-09
  7.22059830e-09 6.79429131e-09 6.40008551e-09 6.03505588e-09
  5.6965

array([1., 1., 1.])